In [2]:
# installing modules 
!pip3 install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 11.0 MB/s eta 0:00:00


In [3]:
import pandas as pd 
import numpy as np


from sklearn. preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

#models for ensemble 
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV


from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from xgboost import XGBClassifier

In [26]:
#data downloading 

train = pd.read_csv('/content/train.csv')
X_test = pd.read_csv('/content/test.csv')


# Fill missing values (Nan) with 0 
train.fillna(0, inplace = True)
X_test.fillna(0 , inplace= True)


In [5]:

# separate y_train / X_train from train 
y_train = train['Transported']
X_train = train.drop(columns = ['Transported'],axis = 1)


X_train


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre


In [6]:
#Let's separate the Name column into a variable and encode the names into unique numbers

name = X_train['Name']
X_train = X_train.drop( columns = ['Name'] , axis = 1 )
labelencoder = LabelEncoder()
name = name.astype('str')
name = labelencoder.fit_transform(name)
name_df = pd.DataFrame(name)

In [7]:
# Split the column "PassengerId" into 2 new columns in ds : Passenger and Number_in_group

passengers = []
ids = []

Pass_id_dict = {}
for i in X_train.PassengerId :
  split_i = i.split('_')
  split_i[1] = int(split_i[1])
  
  passengers.append(split_i[0])
  ids.append(split_i[1])
  
X_train['Group'] = passengers
X_train['Number_in_group'] = ids


In [8]:
#Delete old column from ds 
X_train = X_train.drop( columns = ['PassengerId'] , axis = 1 )


In [9]:
#split the column cabin into 3 new columns : Deck / Num / Side 

deck =[]
num=[]
side=[]

for i in X_train.Cabin :
  if i != 0 :
    
    split_i = i.split('/')
    deck.append(split_i[0])
    num.append(int(split_i[1]))
    side.append(split_i[2])
  if i == 0 :
    deck.append(0)
    num.append(0)
    side.append(0)



  
X_train['Deck'] = deck
X_train['Num'] = num
X_train['Side'] = side

# Delete old one 
X_train = X_train.drop( columns = ['Cabin' ] , axis = 1 )





In [10]:
# Group categorical columns to use OneHotEncoding

categorical = X_train[['HomePlanet' , 'Destination'  , 'Deck' , 'Side' ]]
categorical = pd.get_dummies(categorical)


In [11]:
#Detele columns after One Hot Encoding where unique value is Nan ( 0 after fillna )

categorical = categorical.drop( columns = ['HomePlanet_0' , 'Destination_0' , 'Deck_0' , 'Side_0'] , axis = 1)


In [14]:
# Transform numerical columns to type int 

numerical = X_train[['CryoSleep' , 'Age' , 'VIP' , 'RoomService' , 'FoodCourt' , 'ShoppingMall' , 'Spa' , 'VRDeck' , 'Group' , 'Number_in_group' , 'Num']]
numerical = numerical.astype('int')


# Join cat columns and num columns
X_train = categorical.join(numerical)

In [15]:
# join the columns "Name" too

X_train = X_train.join(name_df)
X_train = X_train.rename( columns = { 0 : 'Name'  })

In [16]:
X_train

,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_A,Deck_B,Deck_C,Deck_D,...,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Group,Number_in_group,Num,Name
0,0,1,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,1,1,0,5253
1,1,0,0,0,0,1,0,0,0,0,...,0,109,9,25,549,44,2,1,0,4503
2,0,1,0,0,0,1,1,0,0,0,...,1,43,3576,0,6715,49,3,1,0,458
3,0,1,0,0,0,1,1,0,0,0,...,0,0,1283,371,3329,193,3,2,0,7150
4,1,0,0,0,0,1,0,0,0,0,...,0,303,70,151,565,2,4,1,1,8320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0,1,0,1,0,0,1,0,0,0,...,1,0,6819,0,1643,74,9276,1,98,3525
8689,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,9278,1,1499,4781
8690,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1872,1,0,9279,1,1500,3003
8691,0,1,0,1,0,0,0,0,0,0,...,0,0,1049,0,353,3235,9280,1,608,1597


In [17]:
# all our DataFrame consists now of digits only

In [18]:
y_train

0       False
1        True
2       False
3       False
4        True
        ...  
8688    False
8689    False
8690     True
8691    False
8692     True
Name: Transported, Length: 8693, dtype: bool

In [19]:
# Transform y_train from bool to int

y_train = y_train.astype('int')



In [20]:
# Using of Standard Scaler to convert all values to a normal distribution with mean 0 and standart deviation 1

scaler = StandardScaler()
names = X_train.columns 
X_train = scaler.fit_transform(X_train)
X_train = pd.DataFrame( X_train , columns=names)
 

In [21]:
#Ансамбль / Ensemble

base_models = [
               ('rf',  RandomForestClassifier(  n_estimators = 100 , criterion = "entropy") ) , 
               ( 'KNN' , KNeighborsClassifier( algorithm = 'ball_tree') ) , 
               ( 'SVC' , SVC() ) ,
               ( 'Bayes' , BernoulliNB() ) , 
               ( 'DecisionTree' , DecisionTreeClassifier() ) , 
               ( 'Xgboost' , XGBClassifier(objective="binary:logistic") ) 
               


               ]
meta_model = CatBoostClassifier( 
    eval_metric = 'Accuracy' , loss_function = 'Logloss' ,
    learning_rate = 0.029999999329447746 ,
    iterations = 200 ,
    custom_metric = 'Accuracy' ,
    verbose = 10 ,
    l2_leaf_reg =3                   )

stacking_model = StackingClassifier(estimators=base_models, 
                                    final_estimator=meta_model, 
                                    passthrough= True , 
                                    cv=5,
                                    verbose=2,
                                    stack_method = 'predict')


# the code of GridSearchCv . Finding the best params of meta classificator and base models. ( Random Forest below )

# params = {
    
#   'rf__bootstrap' : [True , False ] , 
#   'rf__n_estimators' : [ 100 ,  150 ,  200]  ,
#   'rf__criterion' : ['gini' , 'entropy'  , 'log_loss'] 
  

# }

# grid = GridSearchCV(estimator= stacking_model, param_grid=params, cv=2 , scoring = 'accuracy' , verbose = 3)




In [22]:
# train the model

stacking_model.fit(X_train , y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    6.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    8.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent work

0:	learn: 0.7950075	total: 51.5ms	remaining: 10.2s
10:	learn: 0.8034050	total: 90.1ms	remaining: 1.55s
20:	learn: 0.8051306	total: 127ms	remaining: 1.09s
30:	learn: 0.8055907	total: 165ms	remaining: 898ms
40:	learn: 0.8061659	total: 209ms	remaining: 811ms
50:	learn: 0.8078914	total: 255ms	remaining: 744ms
60:	learn: 0.8076613	total: 297ms	remaining: 677ms
70:	learn: 0.8084666	total: 337ms	remaining: 611ms
80:	learn: 0.8093869	total: 381ms	remaining: 560ms
90:	learn: 0.8122627	total: 419ms	remaining: 502ms
100:	learn: 0.8123778	total: 463ms	remaining: 454ms
110:	learn: 0.8124928	total: 501ms	remaining: 401ms
120:	learn: 0.8138732	total: 539ms	remaining: 352ms
130:	learn: 0.8155988	total: 583ms	remaining: 307ms
140:	learn: 0.8169792	total: 621ms	remaining: 260ms
150:	learn: 0.8175544	total: 659ms	remaining: 214ms
160:	learn: 0.8192799	total: 708ms	remaining: 171ms
170:	learn: 0.8218107	total: 750ms	remaining: 127ms
180:	learn: 0.8253767	total: 802ms	remaining: 84.2ms
190:	learn: 0.826412

StackingClassifier(cv=5,
                   estimators=[('rf',
                                RandomForestClassifier(criterion='entropy')),
                               ('KNN',
                                KNeighborsClassifier(algorithm='ball_tree')),
                               ('SVC', SVC()), ('Bayes', BernoulliNB()),
                               ('DecisionTree', DecisionTreeClassifier()),
                               ('Xgboost',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=...
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              n_estimators=100, n_jobs=None,
                                              num_parallel_tree=None,
                                              predictor=None, random_state=None, ...))],
                   final_estimator=<catboost.core.CatBoostClassifier object at 0x7fb97cbc6c40>,
                   passthrough=True, stack_method='predict', verbose=2)

In [27]:
# Use the code we used to prepare the train data for the test data 

passenger_id = X_test['PassengerId'] 

In [28]:
# test
name = X_test['Name']
X_test = X_test.drop(columns =['Name'] , axis = 1 )

labelencoder = LabelEncoder()
name = name.astype('str')
name = labelencoder.fit_transform(name)
name_df = pd.DataFrame(name)

passengers = []
ids = []
Pass_id_dict = {}
for i in X_test.PassengerId :
  split_i = i.split('_')
  split_i[1] = int(split_i[1])
  
  passengers.append(split_i[0])
  ids.append(split_i[1]) 
X_test['Group'] = passengers
X_test['Number_in_group'] = ids

X_test = X_test.drop( columns = ['PassengerId'] , axis = 1 )



import numpy as np
deck =[]
num=[]
side=[]
for i in X_test.Cabin :
  if i != 0 :
    
    split_i = i.split('/')
    deck.append(split_i[0])
    num.append(int(split_i[1]))
    side.append(split_i[2])
  if i == 0 :
    deck.append(0)
    num.append(0)
    side.append(0)



  
X_test['Deck'] = deck
X_test['Num'] = num
X_test['Side'] = side
X_test = X_test.drop( columns = ['Cabin' ] , axis = 1 )

categorical = X_test[['HomePlanet' , 'Destination'  , 'Deck' , 'Side']]
categorical = pd.get_dummies(categorical)




numerical = X_test[['CryoSleep' , 'Age' , 'VIP' , 'RoomService' , 'FoodCourt' , 'ShoppingMall' , 'Spa' , 'VRDeck' , 'Group' , 'Number_in_group' , 'Num']]
categorical = categorical.drop( columns = ['HomePlanet_0' , 'Destination_0' , 'Deck_0' , 'Side_0'] , axis = 1)
numerical = numerical.astype('int')

X_test = categorical.join(numerical)

X_test = X_test.join(name_df)
X_test = X_test.rename( columns = { 0 : 'Name'  })
X_test

,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_A,Deck_B,Deck_C,Deck_D,...,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Group,Number_in_group,Num,Name
0,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,13,1,3,2913
1,1,0,0,0,0,1,0,0,0,0,...,0,0,9,0,2823,0,18,1,4,2407
2,0,1,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,19,1,0,3377
3,0,1,0,0,0,1,0,0,1,0,...,0,0,6652,0,181,585,21,1,1,2712
4,1,0,0,0,0,1,0,0,0,0,...,0,10,0,635,0,0,23,1,5,669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,9266,2,1496,2106
4273,1,0,0,0,0,1,0,0,0,0,...,0,0,847,17,10,144,9269,1,0,2655
4274,0,0,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,9271,1,296,2086
4275,0,1,0,0,0,0,0,0,0,1,...,0,0,2680,0,0,523,9273,1,297,2328


In [29]:
# Standart Scaler for test data

scaler = StandardScaler()
names = X_test_f.columns 
X_test = scaler.fit_transform(X_test_f)
X_test = pd.DataFrame( X_test , columns=names)



In [30]:
X_test


,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_A,Deck_B,Deck_C,Deck_D,...,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Group,Number_in_group,Num,Name
0,0.943382,-0.553131,-0.525314,-0.494734,-0.315862,0.668497,-0.153136,-0.30408,-0.300857,-0.244898,...,-0.132689,-0.357339,-0.283840,-0.312173,-0.267841,-0.246712,-1.703425,-0.489847,-1.146527,0.708477
1,0.943382,-0.553131,-0.525314,-0.494734,-0.315862,0.668497,-0.153136,-0.30408,-0.300857,-0.244898,...,-0.132689,-0.357339,-0.277879,-0.312173,2.287504,-0.246712,-1.701584,-0.489847,-1.144593,0.297314
2,-1.060016,1.807889,-0.525314,2.021290,-0.315862,-1.495894,-0.153136,-0.30408,3.323836,-0.244898,...,-0.132689,-0.357339,-0.283840,-0.312173,-0.267841,-0.246712,-1.701215,-0.489847,-1.152328,1.085511
3,-1.060016,1.807889,-0.525314,-0.494734,-0.315862,0.668497,-0.153136,-0.30408,3.323836,-0.244898,...,-0.132689,-0.357339,4.121518,-0.312173,-0.104002,0.226648,-1.700479,-0.489847,-1.150395,0.545149
4,0.943382,-0.553131,-0.525314,-0.494734,-0.315862,0.668497,-0.153136,-0.30408,-0.300857,-0.244898,...,-0.132689,-0.340723,-0.283840,0.832122,-0.267841,-0.246712,-1.699743,-0.489847,-1.142660,-1.114938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,0.943382,-0.553131,-0.525314,-0.494734,-0.315862,0.668497,-0.153136,-0.30408,-0.300857,-0.244898,...,-0.132689,-0.357339,-0.283840,-0.312173,-0.267841,-0.246712,1.703574,0.492373,1.740519,0.052730
4273,0.943382,-0.553131,-0.525314,-0.494734,-0.315862,0.668497,-0.153136,-0.30408,-0.300857,-0.244898,...,-0.132689,-0.357339,0.277095,-0.281538,-0.258790,-0.130193,1.704679,-0.489847,-1.152328,0.498833
4274,-1.060016,-0.553131,1.903624,2.021290,-0.315862,-1.495894,-0.153136,-0.30408,-0.300857,4.083326,...,-0.132689,-0.357339,-0.283840,-0.312173,-0.267841,-0.246712,1.705415,-0.489847,-0.579947,0.036478
4275,-1.060016,1.807889,-0.525314,-0.494734,-0.315862,-1.495894,-0.153136,-0.30408,-0.300857,4.083326,...,-0.132689,-0.357339,1.491019,-0.312173,-0.267841,0.176479,1.706152,-0.489847,-0.578013,0.233121


In [31]:
#predictions 
y_test = stacking_model.predict(X_test)

In [32]:
result = pd.DataFrame({'PassengerId' : passenger_id , 'Transported' : y_test.astype(bool)})  #convert it into new DF

In [33]:
result

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
...,...,...
4272,9266_02,False
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [34]:
result.to_csv ('/content/our_predictions.csv', index= False ) 